In [1]:
import cdsapi
import os
from tqdm import tqdm
import time  

In [2]:
# Initialize CDS API client
c = cdsapi.Client()

In [3]:
# Define countries and their bounding boxes
countries_dict = {
    "SA": "Saudi Arabia",
    #"YE": "Yemen",
    #"OM": "Oman",
    #"QA": "Qatar",
    #"BH": "Bahrain",
    #"AE": "United Arab Emirates",
    #"KW": "Kuwait"
}

In [4]:
country_areas = {
    "SA": [16.0, 34.5, 32.0, 55.6667], # Saudi Arabia
    #"YE": [12.1111, 41.833, 18.999, 54.5303],  # Yemen
    #"OM": [16.642, 52.0, 26.4, 59.8],  # Oman
    #"QA": [24.4814, 50.757, 26.1546, 51.6369],  # Qatar
    #"BH": [25.5350, 50.454, 26.2640, 50.6640],  # Bahrain
    #"AE": [22.6333, 51.5833, 26.084, 56.3817],  # United Arab Emirates
    #"KW": [28.5243, 46.5527, 30.1037, 48.4315],  # Kuwait

}

In [5]:
# Define variables for surface and pressure levels
surface_variables = [#'2m_temperature',
                    #'2m_dewpoint_temperature',
                    '10m_u_component_of_wind',
                    '10m_v_component_of_wind',
                    'surface_solar_radiation_downwards',
                    #'surface_thermal_radiation_downwards',
                    #'evaporation', 
                    #'potential_evaporation',
                    #'sea_surface_temperature',
                    #'soil_temperature_level_1',
                    #'soil_temperature_level_2',
                    #'soil_temperature_level_3',
                    #'soil_temperature_level_4',
                    #'volumetric_soil_water_layer_1',
                    #'volumetric_soil_water_layer_2',
                    #'volumetric_soil_water_layer_3',
                    #'volumetric_soil_water_layer_4',
                    #'leaf_area_index_high_vegetation',
                    #'leaf_area_index_low_vegetation',
                    #'surface_pressure',
                    #'mean_sea_level_pressure',
                    #'convective_available_potential_energy',
                    #'geopotential',
                    #'instantaneous_10m_wind_gust',
                    #'total_precipitation',
                    #'k_index'
                    ]

pressure_level_variables = [#'geopotential', 
                            #'relative_humidity', 
                            #'temperature', 
                            #'u_component_of_wind', 
                            #'v_component_of_wind', 
                            #'vertical_velocity', 
                            #'vorticity'
                            ]


In [6]:
# Output directory
output_directory = r"Z:\Thesis\Data\Met\ERA5"

In [7]:
def download_data(year, country_code, country_name, area, output_directory):
    for variable_type, variables in [("surface", surface_variables), ("pressure", pressure_level_variables)]:
        output_path = os.path.join(output_directory, year, country_name, variable_type)
        os.makedirs(output_path, exist_ok=True)

        for variable in variables:
            file_name = f"{year}_{country_name}_{variable_type}_{variable}.nc"
            request = {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'area': area,
                'variable': variable,
                'year': year,
                'month': [f"{i:02d}" for i in range(1, 13)],
                'day': [f"{i:02d}" for i in range(1, 31)],
                'time': [f"{i:02d}:00" for i in range(0, 24, 3)],
            }
            if variable_type == 'pressure':
                request.update({'pressure_level': ['1000', '925', '850', '700', '500', '300', '200', '100', '50', '10']})

            print(f"Downloading {file_name} for {country_name}, {year}...")
            c.retrieve(
                f'reanalysis-era5-{"single-levels" if variable_type == "surface" else "pressure-levels"}',
                request,
                os.path.join(output_path, file_name)
            )

In [8]:
# Loop through each year from 2019 to 2022
for year in range(2013, 2014):
    # Loop through each country in the dictionary
    for country_code, country_name in countries_dict.items():
        area = country_areas[country_code]
        # Initialize the progress bar
        with tqdm(total=len(surface_variables + pressure_level_variables),
                  desc=f"Downloading data for {country_name} {year}") as pbar:
            retry_count = 0
            success = False
            while retry_count < 100 and not success:
                try:
                    # Call the download function
                    download_data(str(year), country_code, country_name, area, output_directory)
                    # Update the progress bar after each completion
                    pbar.update(len(surface_variables + pressure_level_variables))
                    success = True
                except Exception as e:
                    retry_count += 1
                    print(f"Error downloading data for {country_name} {year}: {e}")
                    print(f"Retrying {retry_count}/100...")
                    time.sleep(5)  # Wait for 5 seconds before retrying
            if not success:
                print(f"Failed to download data for {country_name} {year} after 3 attempts.")
                break  # Stop the script if retries fail

2025-02-08 20:56:20,380 INFO Request ID is d18aeb78-d308-43e9-8516-456c31ce7b7b
2025-02-08 20:56:20,592 INFO status has been updated to accepted
2025-02-08 21:31:29,711 INFO status has been updated to running
2025-02-08 21:35:31,572 INFO status has been updated to successful


477e24d6d821fd4d1f4adb70d2dc35c9.nc:   0%|          | 0.00/49.1M [00:00<?, ?B/s]

2025-02-08 21:35:45,655 INFO Request ID is 9a40e0a3-a825-4e89-b984-5a6a81c0b3d2
2025-02-08 21:35:45,938 INFO status has been updated to accepted
2025-02-08 21:54:12,506 INFO status has been updated to running
2025-02-08 21:58:19,601 INFO status has been updated to successful


387cbd8599fa8bc46d46b31e9f547717.nc:   0%|          | 0.00/46.6M [00:00<?, ?B/s]

2025-02-08 21:58:27,890 INFO Request ID is 5c663afc-63f3-455c-b5ce-2e6345f76698
2025-02-08 21:58:28,137 INFO status has been updated to accepted
2025-02-08 23:13:58,573 INFO status has been updated to running
2025-02-08 23:18:00,273 INFO status has been updated to successful


d68a8c3f34e4ff9bf55808b5618add68.nc:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

In [ ]:
# Loop through each country and download data
for country_code, country_name in countries_dict.items():
    area = country_areas[country_code]
    with tqdm(total=len(surface_variables + pressure_level_variables), desc=f"Downloading {country_name}") as pbar:
        download_data("2022", country_code, country_name, area, output_directory)
        pbar.update(len(surface_variables + pressure_level_variables))

In [ ]:
# Loop through each year from 2010 to 2015
for year in range(2016, 2023):
     #Loop through each country in the dictionary
    for country_code, country_name in countries_dict.items():
        area = country_areas[country_code]
         #Initialize the progress bar
        with tqdm(total=len(surface_variables + pressure_level_variables),
                  desc=f"Downloading data for {country_name} {year}") as pbar:
             #Call the download function
            download_data(str(year), country_code, country_name, area, output_directory)
            # Update the progress bar after each completion
            pbar.update(len(surface_variables + pressure_level_variables))
